# Part 2: Plotting and Basic Spatial Metrics

**Tutor:** Tim Treis
**Time:** 35 minutes

---

Now that we understand what a `SpatialData` object is and how to explore it interactively, let's learn how to create static, publication-quality plots and ask our first spatial question: "Is this gene's expression pattern random, or is it spatially organized?"

**Goals:**
1. Use `spatialdata-plot` to create layered, static images.
2. Introduce a different technology: spot-based Visium data.
3. Use `squidpy` to calculate and visualize a key spatial statistic, Moran's I.

### Setup
First, let's import the libraries we'll need for this notebook.

In [ ]:
# For cleaner output

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import spatialdata as sd
import matplotlib.pyplot as plt
import scanpy as sc
import squidpy as sq
from pathlib import Path

# Define the path to our data directory
# Note: This path is relative to the repository's root directory
_DATA_DIR_PATH = Path("../data/")
_VISIUM_PATH = _DATA_DIR_PATH / "visium_glioblastoma_subset.zarr"
_XENIUM_PATH = _DATA_DIR_PATH / "xenium_lung_cancer_subset.zarr"

### Static Plotting with `spatialdata-plot`

For this section, we will use a dataset from a 10x Genomics Visium experiment. Unlike the high-resolution Xenium data from the previous notebook, Visium is a **spot-based** technology. It captures the whole transcriptome, but at the resolution of spots (~55µm) which may contain multiple cells.

![Visium technology](../resources/visium_tech.png)

Let's load our pre-processed Visium dataset.

In [ ]:
sdata_visium = sd.read_zarr(_VISIUM_PATH)
sdata_visium

In [ ]:
# The following command will fail because we haven't loaded spatialdata-plot yet

sdata_visium.pl.render_images().pl.show()

The `spatialdata-plot` library adds a `.pl` accessor to our `SpatialData` object. We can chain `render_*` functions to build up a plot layer by layer, similar to `ggplot`.

## .pl.render_images() and .pl.show()

The [`.pl.render_images()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_images) function allows you to display the contaiend images. Under-the-hood optimizations allow you plot even extremely large images quickly. Furthermore, optional coordinate system transformations are respected.

The [`.pl.show()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.show) function then actually renders the image, taking all previous function calls into account. They are evaluated in order and then stacked on top of each other. Only calling any of the `.pl.render_X` functions without `.pl.show` will not yield an image.

Let's start by loading the library so the `.pl` acessor becomes available. We'll test it by rendering the histology image.

In [ ]:
import spatialdata_plot as sdp

sdata_visium.pl.render_images().pl.show()

We see that the same image gets shown twice. That's because the Visium `SpatialData` object contains two coordinate systems: `downscaled_lowres` and `downscaled_hires`. Typically, the `global` coordinate system would contain the full-resolution image, but we've removed this here for faster downloading of the data.

We can select one specific coordinate system in the `pl.show` function.

In [ ]:
sdata_visium.pl.render_images().pl.show(coordinate_systems="downscaled_hires")

We can further control the size of the figure by passing either the `figsize` parameter to the `pl.show` function or by directly passing an `ax` object.

In [ ]:
sdata_visium.pl.render_images().pl.show(coordinate_systems="downscaled_hires", figsize=(2, 2))

fig, ax = plt.subplots(figsize=(3, 3))
sdata_visium.pl.render_images().pl.show(coordinate_systems="downscaled_hires", ax=ax)

We can adjust the title of the figure (default: name of the coordinate system) as well.

In [ ]:
(
    sdata_visium
        .pl.render_images()
        .pl.show(
            coordinate_systems="downscaled_hires",
            figsize=(2, 2),
            title="Visium",
        )
)

Within the actual [`.pl.render_images()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_images) call, we can also plot only selective channels. This becomes useful when, for example, plotting multi-channel IF data.(
    sdata_visium
        .pl.render_images()
        .pl.show(
            coordinate_systems="downscaled_hires",
            figsize=(2, 2),
            title="Visium",
        )
)

In [ ]:
(
    sdata_visium
        .pl.render_images(channel=1)  # The green channel of the RGB image
        .pl.show(
            coordinate_systems="downscaled_hires",
            figsize=(3, 2),
            title="Visium",
        )
)

## .pl.render_shapes()

Now, let's overlay the circular Visium spots (`Shapes`) on top of the image.

In [ ]:
(
    sdata_visium
        .pl.render_images()
        .pl.render_shapes()  # First image, then shape, so that they're visible
        .pl.show(
            coordinate_systems="downscaled_hires",
            title="Visium",
        )
)

We can make them semi-transparent to see the tissue underneath.

In [ ]:
(
    sdata_visium
        .pl.render_images()
        .pl.render_shapes(fill_alpha=0.2)
        .pl.show(
            coordinate_systems="downscaled_hires",
            title="Visium",
        )
)

However, we can of course also cover them by certain covariates such as the expression of certain genes.

In [ ]:
(
    sdata_visium
        .pl.render_images()
        .pl.render_shapes(color="CST3")
        .pl.show(
            coordinate_systems="downscaled_hires",
            title="Visium",
        )
)

In the case of Visium data, we can furthermore specify the shape to be `visium_hex` which then fully covers the tissue and usually leads to a nicer visualization.

In [ ]:
(
    sdata_visium
        .pl.render_images()
        .pl.render_shapes(color="CST3", shape="visium_hex")
        .pl.show(
            coordinate_systems="downscaled_hires",
            title="Visium",
        )
)

We can pass regular [`matplotlib.colors.Normalize`](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Normalize.html) and [`matplotlib.colors.Colormap`](https://matplotlib.org/stable/api/_as_gen/matplotlib.colors.Colormap.html) objects to the function to further modify the plot.

In [ ]:
import matplotlib

norm = matplotlib.colors.Normalize(vmin=0, vmax=400)
cmap = matplotlib.cm.get_cmap("Reds")

(
    sdata_visium
        .pl.render_images()
        .pl.render_shapes(color="CST3", shape="visium_hex", norm=norm, cmap=cmap)  # Alternatively, you can directly pass "Reds" to `cmap`
        .pl.show(
            coordinate_systems="downscaled_hires",
            title="Visium",
        )
)

## .pl.render_shapes() and .pl.render_labels()

We will switch to the Xenium dataset to demonstrate the use of [`.pl.render_shapes`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_shapes) and [`.pl.render_labels`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_labels). With it we can, for example, plot individual cells.

In [ ]:
sdata_xenium = sd.read_zarr(_XENIUM_PATH)

# Let's subset the image to a smaller crop of it so we can better see the changes we're making.
sdata_xenium_subset = sdata_xenium.query.bounding_box(
    axes=["x", "y"],
    min_coordinate=[31_000, 8_500],
    max_coordinate=[34_000, 11_500],
    target_coordinate_system="global",
)
sdata_xenium_subset

In [ ]:
(
    sdata_xenium_subset
        .pl.render_images(elements="he_image")
        .pl.render_labels(elements="cell_labels")
        .pl.show(
            title="Xenium cell segmentations",
            figsize=(12, 12),
        )
)

You can also modify the plotting call to just highlight where th(
    sdata_xenium_subset
        .pl.render_images(elements="he_image")
        .pl.render_labels(elements="cell_labels")
        .pl.show(
            title="Xenium cell segmentations",
            figsize=(12, 12),
        )
)e segmentation masks are without obfuscating the H&E images.

In [ ]:
(
    sdata_xenium_subset
        .pl.render_images(elements="he_image")
        .pl.render_labels(
            elements="cell_labels",
            fill_alpha=0,
            outline_alpha=1,
            contour_px=3,
        ).pl.show(
            title="Xenium cell segmentations",
            figsize=(12, 12),
        )
)

We see several things:
1) The underlaying histopathology image is fairly low in resolution and gets automatically upscaled to align with the segmentation masks.
2) By far not every cell is succesfully segmented.
3) The cell segmentation masks are colored with random colors - this is because the contained `AnnData` object doesn't annotate them. We can check this with:

In [ ]:
sdata_xenium_subset.tables["table"].uns["spatialdata_attrs"]

We see that the table annotates the `cell_circles` element. For Xenium, `spatialdata-io` automatically converts cell labels to circles. This is a performance improvement since Xenium slides can easily contain more than 500k cells. 

So, let's render these **Shapes** instead:

In [ ]:
fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(7, 3))

(
    sdata_xenium_subset
        .pl.render_images("he_image")
        .pl.render_shapes("cell_circles", color="transcript_counts")
        .pl.show(
            title="transcript counts",
            ax=axs[0],
        )
)

(
    sdata_xenium_subset
        .pl.render_images("he_image")
        .pl.render_shapes("cell_circles", color="cell_area")
        .pl.show(
            title="cell area",
            ax=axs[1],
        )
)

fig.tight_layout()

With this plot we can, for example, easily see that transcript count primarily correlates to cell area.

<div style="border: 1px solid #FF5C00; border-left-width: 15px; padding: 10px; background-color: #FFA500; color: black;">
    <strong>Note:</strong>
    <p>In the following, we'll modify what the AnnData table annotated. This is slightly more complicated, but usually not neccecary. However, one can still benefit from knowing about this option.</p>
</div>

In [ ]:
sdata_xenium_subset.tables["table"].obs["region"] = "cell_labels"

sdata_xenium_subset.set_table_annotates_spatialelement(
    table_name="table",
    region="cell_labels",
    region_key="region",
    instance_key='cell_labels',
)

Now that we've adjusted this, we can also color the labels.

In [ ]:
(
    sdata_xenium_subset
        .pl.render_images(elements="he_image")
        .pl.render_labels(elements="cell_labels", color="cell_area")
        .pl.show(
            title="Xenium cell segmentations",
        )
)

## .pl.render_points()

We can also directly visualize the transcript localisations that would otherwise we aggregated to a cell x gene matrix in Xenium. We'll use [`.pl.render_points()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_points) for that. For visual guidance, we'll overlay the cell segmentation masks.

Due to the way the data is subset, one of the performance optimisations (the use of [`DataShader`](https://datashader.org/)) actually results in a weird visualization, so we'll disable it.

In [ ]:
(
    sdata_xenium_subset
        .pl.render_points(
            method="matplotlib", # We don't want an aggregated version but the raw points (-> no datashader)
        )  
        .pl.render_labels(
            elements="cell_labels",
            fill_alpha=0,
            outline_alpha=1,
            contour_px=3,
        ).pl.show(
            title="Xenium points",
            figsize=(12, 12),
        )
)

Similar to the other function, [`.pl.render_points()`](https://spatialdata.scverse.org/projects/plot/en/latest/plotting.html#spatialdata_plot.pl.basic.PlotAccessor.render_points) gives us the option to color the points by certain covariates. Here, we're coloring by the fact whether a transcript was localized inside the nucleus mask or not. Of note is that this information isn't stored in the `AnnData` table, but `spatialdata-plot` automatically identifies the correct data source.

In [ ]:
(
    sdata_xenium_subset
        .pl.render_points(
            method="matplotlib", # We don't want an aggregated version but the raw points (-> no datashader)
            color="overlaps_nucleus"
        )  
        .pl.render_labels(
            elements="cell_labels",
            fill_alpha=0,
            outline_alpha=1,
            contour_px=3,
        ).pl.show(
            title="Xenium points",
            figsize=(12, 12),
        )
)

In case we're interested in specific genes, we can also only highlight those.

In [ ]:
(
    sdata_xenium_subset
        .pl.render_points(
            method="matplotlib", # We don't want an aggregated version but the raw points (-> no datashader)
            color="feature_name",
            groups=["KRT7", "MLPH", "AGR3"],
            palette=["red", "green", "blue"],
            size=5,
        )  
        .pl.render_labels(
            elements="cell_labels",
            fill_alpha=0,
            outline_alpha=1,
            contour_px=3,
        ).pl.show(
            title="Xenium points",
            figsize=(12, 12),
        )
)

Lastly, let's combine several things we learned to create a more complicated plot.

In [ ]:

# make custom cmap and norm
cmap = matplotlib.cm.get_cmap("Reds").copy()
cmap.set_under((0.0, 0.0, 0.0, 0.0))
norm = matplotlib.colors.Normalize(vmin=5, vmax=None)

fig, axs = plt.subplots(ncols=2, nrows=1, figsize=(12, 4.5))

(
    sdata_xenium_subset
        .pl.render_points(
            method="matplotlib", # We don't want an aggregated version but the raw points (-> no datashader)
            color="feature_name",
            groups=["KRT7"],
            palette=["red"],
            size=1,
        )  
        .pl.render_labels(
            elements="cell_labels",
            fill_alpha=0,
            outline_alpha=1,
            contour_px=3,
        ).pl.show(
            title="KRT7 transcript localizations",
            ax=axs[0],
        )
)

(
    sdata_xenium_subset
        .pl.render_images("he_image")
        .pl.render_labels(
            elements="cell_labels",
            color="KRT7",
            fill_alpha=0.5,
            outline_alpha=1,
            cmap="Reds",
        ).pl.show(
            title="Aggregated KRT7 counts",
            ax=axs[1],
        )
)

fig.tight_layout()

<div style="border: 1px solid #4CAF50; border-left-width: 15px; padding: 10px; background-color: #F0FFF0; color: black;">
    <strong>Summary:</strong>
    <p>This concludes the second part of our workshop. We've learned how to load and interact with different spatial technologies and how to perform basic plotting. In the next part, Anthony will guide us through a complete downstream analysis, including cell type clustering and integrating data from multiple experiments.</p>
</div>